### Advanced Data Transformation with Polars

The examples below use `.head()` to reduce the output to a few rows and take up less space.
If you want the full output, remove `.head()` from the code.

This notebooked is divided into sections. If your code editor supports it, you can use the **Outline** functionality to easily go to the code section you are interested in.

For more details on Polars functions, check out the Polars API reference: https://pola-rs.github.io/polars/py-polars/html/reference/index.html

In [2]:
import polars as pl

# Configure the number of characters to show for each string column
pl.Config.set_fmt_str_lengths(30)

polars.config.Config

In [3]:
pokemon = pl.read_parquet('../datasets/pokemon_simplified.parquet')

In [4]:
pokemon.head(2)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary
str,str,str,list[str],i64,i64,i64,i64,i64,i64
"""Bulbasaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",45,49,49,45,1,0
"""Ivysaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",60,62,63,60,1,0


#### Conditions

In [5]:
# Identify the top 25% of Pokemon based on the attack stats
# Use the quantile function to get the 75th percentile

pokemon.with_columns(
    attack_category = pl.when(pl.col('attack') > pl.col('attack').quantile(0.75)).then('Top 25%')
).head(8)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary,attack_category
str,str,str,list[str],i64,i64,i64,i64,i64,i64,str
"""Bulbasaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",45,49,49,45,1,0,null
"""Ivysaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",60,62,63,60,1,0,null
"""Venusaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",80,100,123,80,1,0,null
"""Charmander""","""fire""",null,"[""Blaze"", ""Solar Power""]",39,52,43,65,1,0,null
"""Charmeleon""","""fire""",null,"[""Blaze"", ""Solar Power""]",58,64,58,80,1,0,null
"""Charizard""","""fire""","""flying""","[""Blaze"", ""Solar Power""]",78,104,78,100,1,0,"""Top 25%"""
"""Squirtle""","""water""",null,"[""Torrent"", ""Rain Dish""]",44,48,65,43,1,0,null
"""Wartortle""","""water""",null,"[""Torrent"", ""Rain Dish""]",59,63,80,58,1,0,null


In [6]:
# Identify the top 25% of Pokemon based on the attack stats
# Use otherwise to assign the remaining 75% of Pokemon to the 'Other' category

pokemon.with_columns(
    attack_category = pl.when(pl.col('attack') > pl.col('attack').quantile(0.75)).then('Top 25%').otherwise('Other')
).head(8)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary,attack_category
str,str,str,list[str],i64,i64,i64,i64,i64,i64,str
"""Bulbasaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",45,49,49,45,1,0,"""Other"""
"""Ivysaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",60,62,63,60,1,0,"""Other"""
"""Venusaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",80,100,123,80,1,0,"""Other"""
"""Charmander""","""fire""",null,"[""Blaze"", ""Solar Power""]",39,52,43,65,1,0,"""Other"""
"""Charmeleon""","""fire""",null,"[""Blaze"", ""Solar Power""]",58,64,58,80,1,0,"""Other"""
"""Charizard""","""fire""","""flying""","[""Blaze"", ""Solar Power""]",78,104,78,100,1,0,"""Top 25%"""
"""Squirtle""","""water""",null,"[""Torrent"", ""Rain Dish""]",44,48,65,43,1,0,"""Other"""
"""Wartortle""","""water""",null,"[""Torrent"", ""Rain Dish""]",59,63,80,58,1,0,"""Other"""


In [7]:
# Identify the top 25% of Pokemon based on the attack stats AND the bottom 25%
# Use multiple when then statements

pokemon.with_columns(
    attack_category = 
        pl.when(pl.col('attack') > pl.col('attack').quantile(0.75)).then('Top 25%')
        .when(pl.col('attack') < pl.col('attack').quantile(0.25)).then('Bottom 25%')
        .otherwise('Other')
).head(8)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary,attack_category
str,str,str,list[str],i64,i64,i64,i64,i64,i64,str
"""Bulbasaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",45,49,49,45,1,0,"""Bottom 25%"""
"""Ivysaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",60,62,63,60,1,0,"""Other"""
"""Venusaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",80,100,123,80,1,0,"""Other"""
"""Charmander""","""fire""",null,"[""Blaze"", ""Solar Power""]",39,52,43,65,1,0,"""Bottom 25%"""
"""Charmeleon""","""fire""",null,"[""Blaze"", ""Solar Power""]",58,64,58,80,1,0,"""Other"""
"""Charizard""","""fire""","""flying""","[""Blaze"", ""Solar Power""]",78,104,78,100,1,0,"""Top 25%"""
"""Squirtle""","""water""",null,"[""Torrent"", ""Rain Dish""]",44,48,65,43,1,0,"""Bottom 25%"""
"""Wartortle""","""water""",null,"[""Torrent"", ""Rain Dish""]",59,63,80,58,1,0,"""Other"""


#### Convert columns to dummies

In [8]:
# Convert the column type1 to dummies

pokemon.head(8).to_dummies(columns=['type1'])

name,type1_fire,type1_grass,type1_water,type2,abilities,hp,attack,defense,speed,generation,is_legendary
str,u8,u8,u8,str,list[str],i64,i64,i64,i64,i64,i64
"""Bulbasaur""",0,1,0,"""poison""","[""Overgrow"", ""Chlorophyll""]",45,49,49,45,1,0
"""Ivysaur""",0,1,0,"""poison""","[""Overgrow"", ""Chlorophyll""]",60,62,63,60,1,0
"""Venusaur""",0,1,0,"""poison""","[""Overgrow"", ""Chlorophyll""]",80,100,123,80,1,0
"""Charmander""",1,0,0,null,"[""Blaze"", ""Solar Power""]",39,52,43,65,1,0
"""Charmeleon""",1,0,0,null,"[""Blaze"", ""Solar Power""]",58,64,58,80,1,0
"""Charizard""",1,0,0,"""flying""","[""Blaze"", ""Solar Power""]",78,104,78,100,1,0
"""Squirtle""",0,0,1,null,"[""Torrent"", ""Rain Dish""]",44,48,65,43,1,0
"""Wartortle""",0,0,1,null,"[""Torrent"", ""Rain Dish""]",59,63,80,58,1,0


#### Grouping and aggregating data

In [9]:
# Group by type and calculate median stats
# Group by is run in parallel so the order of the rows can change

pokemon.groupby('type1').agg(
    pl.col('hp', 'attack', 'defense', 'speed').median()
).head(3)

type1,hp,attack,defense,speed
str,f64,f64,f64,f64
"""water""",68.0,70.0,70.0,64.0
"""grass""",65.0,68.0,66.0,55.0
"""fairy""",76.0,58.5,67.5,47.0


In [10]:
# Group by type and calculate median stats
# Maintaining the order is possible but it is slower

pokemon.groupby('type1', maintain_order=True).agg(
    pl.col('hp', 'attack', 'defense', 'speed').median()
).head(3)

type1,hp,attack,defense,speed
str,f64,f64,f64,f64
"""grass""",65.0,68.0,66.0,55.0
"""fire""",65.0,79.5,61.5,72.5
"""water""",68.0,70.0,70.0,64.0


In [11]:
# Group on multiple columns works the same way

pokemon.groupby(['type1', 'is_legendary']).agg(
    pl.col('hp', 'attack', 'defense', 'speed').median().round(0)
).head(3)

type1,is_legendary,hp,attack,defense,speed
str,i64,f64,f64,f64,f64
"""water""",0,65.0,70.0,69.0,60.0
"""rock""",0,66.0,84.0,95.0,50.0
"""dark""",0,65.0,85.0,68.0,68.0


In [12]:
# Lots of aggregations functions are available

pokemon.groupby('type1').agg(
    avg_attack = pl.col('attack').mean(),
    median_attack = pl.col('attack').median(),
    min_attack = pl.col('attack').min(),
    max_attack = pl.col('attack').max(),
    quantile_25 = pl.col('attack').quantile(0.25),
    quantile_75 = pl.col('attack').quantile(0.75),
    count = pl.count(), # Count the number of pokemon in each group
    first_pokemon = pl.first('name') # Get the first pokemon in each group
).head(3)

type1,avg_attack,median_attack,min_attack,max_attack,quantile_25,quantile_75,count,first_pokemon
str,f64,f64,i64,i64,f64,f64,u32,str
"""ghost""",72.740741,66.0,30,165,50.0,92.0,27,"""Gastly"""
"""psychic""",65.566038,55.0,20,165,35.0,85.0,53,"""Abra"""
"""flying""",66.666667,70.0,30,100,30.0,100.0,3,"""Tornadus"""


In [13]:
# Possible to use Numpy universal functions (ufuncs) in aggregations

import numpy as np

pokemon.groupby('type1').agg(
    sqrt_attack_mean = np.sqrt(pl.col('attack')).mean() # sqrt is the square root function
).head(3)

type1,sqrt_attack_mean
str,f64
"""fighting""",9.847137
"""grass""",8.416553
"""bug""",8.086296


In [14]:
# If aggregation functions are not specified, the result if aggregated in a list

pokemon.groupby('type1').agg(
    pl.col('name')
).head(3)

type1,name
str,list[str]
"""psychic""","[""Abra"", ""Kadabra"", … ""Necrozma""]"
"""fighting""","[""Mankey"", ""Primeape"", … ""Passimian""]"
"""electric""","[""Pikachu"", ""Raichu"", … ""Xurkitree""]"


In [15]:
# Concatenate the names of the pokemon into a string

pokemon.groupby('type1').agg(
    pl.col('name').str.concat(', ')
).head(3)

type1,name
str,str
"""grass""","""Bulbasaur, Ivysaur, Venusaur,…"
"""flying""","""Tornadus, Noibat, Noivern"""
"""fairy""","""Clefairy, Clefable, Cleffa, T…"


In [16]:
# Aggregation with conditions
# Calculate the average attack for Pokemons with a speed above 100, by type

pokemon.groupby('type1').agg(
    avg_attack_when_speed_above_100 = pl.col('attack').filter(pl.col('speed') > 100).mean()
).head(3)


type1,avg_attack_when_speed_above_100
str,f64
"""normal""",98.0
"""dark""",105.428571
"""electric""",79.25


In [17]:
# We can also create functions to use in aggregations

def avg_attack_when_speed_above_threshold(speed):
    return pl.col('attack').filter(pl.col('speed') > speed).mean().alias(f'avg_attack_when_speed_above_{speed}')

pokemon.groupby('type1').agg(
    avg_attack_when_speed_above_threshold(speed) for speed in range(20,100,20)
).head(3)

type1,avg_attack_when_speed_above_20,avg_attack_when_speed_above_40,avg_attack_when_speed_above_60,avg_attack_when_speed_above_80
str,f64,f64,f64,f64
"""dark""",87.892857,90.038462,94.526316,105.9
"""normal""",76.66,78.333333,84.859649,92.228571
"""rock""",91.219512,97.96875,108.6875,118.857143


In [18]:
# Show the top 3 pokemon by attack in each group
# We sort before aggregation to make sure we get the top 3
(
    pokemon
    .sort(by='attack', descending=True)
    .groupby('type1').agg(
        top_3_pokemon_by_attack = pl.col('name').head(3) # head(3) filters top 3 pokemons per type
    )
).head(5) # head(5) filters 5 types

type1,top_3_pokemon_by_attack
str,list[str]
"""fairy""","[""Xerneas"", ""Granbull"", ""Snubbull""]"
"""rock""","[""Rampardos"", ""Tyranitar"", ""Diancie""]"
"""ice""","[""Mamoswine"", ""Beartic"", ""Glalie""]"
"""water""","[""Gyarados"", ""Swampert"", ""Kyogre""]"
"""normal""","[""Slaking"", ""Regigigas"", ""Lopunny""]"


In [19]:
# Show the top 3 pokemon by attack and the top 3 pokemon by health in each group
(
    pokemon
    .groupby('type1').agg(
        top_3_pokemon_by_attack = pl.col('name').sort_by('attack', descending=True).head(3),
        top_3_pokemon_by_defense = pl.col('name').sort_by('defense', descending=True).head(3) 
    )
).head(5)

type1,top_3_pokemon_by_attack,top_3_pokemon_by_defense
str,list[str],list[str]
"""fighting""","[""Lucario"", ""Conkeldurr"", ""Crabominable""]","[""Hitmontop"", ""Conkeldurr"", ""Passimian""]"
"""fairy""","[""Xerneas"", ""Granbull"", ""Snubbull""]","[""Togekiss"", ""Xerneas"", ""Comfey""]"
"""water""","[""Gyarados"", ""Swampert"", ""Kyogre""]","[""Slowbro"", ""Cloyster"", ""Carracosta""]"
"""normal""","[""Slaking"", ""Regigigas"", ""Lopunny""]","[""Audino"", ""Arceus"", ""Regigigas""]"
"""ground""","[""Groudon"", ""Landorus"", ""Rhyperior""]","[""Groudon"", ""Rhyperior"", ""Gliscor""]"


In [20]:
# Show the top 3 pokemon by attack and the top 3 pokemon by health without grouping by type

pokemon.select(
    top_3_pokemon_by_attack = pl.col('name').sort_by('attack', descending=True).head(3).implode(),
    top_3_pokemon_by_defense = pl.col('name').sort_by('defense', descending=True).head(3).implode()
)

top_3_pokemon_by_attack,top_3_pokemon_by_defense
list[str],list[str]
"[""Heracross"", ""Kartana"", ""Groudon""]","[""Steelix"", ""Shuckle"", ""Aggron""]"


In [21]:
# I can iterate over the groups and do any action on them

for name, data in pokemon.groupby("is_legendary"):
    print(name)
    print(data.head(2))

1
shape: (2, 10)
┌──────────┬──────────┬────────┬─────────────────┬───┬─────────┬───────┬────────────┬──────────────┐
│ name     ┆ type1    ┆ type2  ┆ abilities       ┆ … ┆ defense ┆ speed ┆ generation ┆ is_legendary │
│ ---      ┆ ---      ┆ ---    ┆ ---             ┆   ┆ ---     ┆ ---   ┆ ---        ┆ ---          │
│ str      ┆ str      ┆ str    ┆ list[str]       ┆   ┆ i64     ┆ i64   ┆ i64        ┆ i64          │
╞══════════╪══════════╪════════╪═════════════════╪═══╪═════════╪═══════╪════════════╪══════════════╡
│ Articuno ┆ ice      ┆ flying ┆ ["Pressure",    ┆ … ┆ 100     ┆ 85    ┆ 1          ┆ 1            │
│          ┆          ┆        ┆ "Snow Cloak"]   ┆   ┆         ┆       ┆            ┆              │
│ Zapdos   ┆ electric ┆ flying ┆ ["Pressure",    ┆ … ┆ 85      ┆ 100   ┆ 1          ┆ 1            │
│          ┆          ┆        ┆ "Static"]       ┆   ┆         ┆       ┆            ┆              │
└──────────┴──────────┴────────┴─────────────────┴───┴─────────┴───────┴──

#### Window functions

In [22]:
# For each pokemon show its attack and the average attack for pokemons of the same type 1

pokemon.select(
    'name', 'type1', 'attack',
    avg_attack_for_same_type1 = pl.mean('attack').over('type1')
).head(5)

name,type1,attack,avg_attack_for_same_type1
str,str,i64,f64
"""Bulbasaur""","""grass""",49,73.769231
"""Ivysaur""","""grass""",62,73.769231
"""Venusaur""","""grass""",100,73.769231
"""Charmander""","""fire""",52,81.5
"""Charmeleon""","""fire""",64,81.5


In [23]:
# For each pokemon show its attack and the average attack for pokemons of the same type 1 and pokemons of the same type 2

pokemon.select(
    'name', 'type1', 'type2', 'attack',
    avg_attack_for_same_type1 = pl.mean('attack').over('type1'),
    avg_attack_for_same_type2 = pl.mean('attack').over('type2')
).head(5)

name,type1,type2,attack,avg_attack_for_same_type1,avg_attack_for_same_type2
str,str,str,i64,f64,f64
"""Bulbasaur""","""grass""","""poison""",49,73.769231,67.617647
"""Ivysaur""","""grass""","""poison""",62,73.769231,67.617647
"""Venusaur""","""grass""","""poison""",100,73.769231,67.617647
"""Charmander""","""fire""",null,52,81.5,74.231771
"""Charmeleon""","""fire""",null,64,81.5,74.231771


In [24]:
# For each pokemon, show the top 3 pokemon by attack of the same type

pokemon.select(
    'name', 'type1', 'attack',
    top_3_attack_pokemon_same_type = pl.col('name').sort_by('attack', descending=True).head(3).implode().over('type1')
).head(5)

name,type1,attack,top_3_attack_pokemon_same_type
str,str,i64,list[str]
"""Bulbasaur""","""grass""",49,"[""Kartana"", ""Abomasnow"", ""Breloom""]"
"""Ivysaur""","""grass""",62,"[""Kartana"", ""Abomasnow"", ""Breloom""]"
"""Venusaur""","""grass""",100,"[""Kartana"", ""Abomasnow"", ""Breloom""]"
"""Charmander""","""fire""",52,"[""Blaziken"", ""Flareon"", ""Ho-Oh""]"
"""Charmeleon""","""fire""",64,"[""Blaziken"", ""Flareon"", ""Ho-Oh""]"


In [25]:
# Show top 3 pokemon by attack and defense

pokemon.select(
    pl.col('type1').head(3).implode().over('type1').flatten(),
    # flatten changes 2D lists (each row has a list of 3 pokemons) to 1D array (each pokemon in a separate row)
    top_3_attack = pl.col('name').sort_by('attack', descending=True).head(3).implode().over('type1').flatten(), 
    top_3_defense = pl.col('name').sort_by('defense', descending=True).head(3).implode().over('type1').flatten()
).head(6)

type1,top_3_attack,top_3_defense
str,str,str
"""grass""","""Kartana""","""Ferrothorn"""
"""grass""","""Abomasnow""","""Kartana"""
"""grass""","""Breloom""","""Leafeon"""
"""fire""","""Blaziken""","""Torkoal"""
"""fire""","""Flareon""","""Turtonator"""
"""fire""","""Ho-Oh""","""Magcargo"""


In [26]:
# Rank the pokemon by attack and defense by type

pokemon.select(
    'name', 'type1', 'attack', 'defense',
    attack_rank = pl.col('attack').rank(method='ordinal', descending=True).over('type1'),
    defense_rank = pl.col('defense').rank(method='ordinal', descending=True).over('type1')
).head(6)

name,type1,attack,defense,attack_rank,defense_rank
str,str,i64,i64,u32,u32
"""Bulbasaur""","""grass""",49,49,61,62
"""Ivysaur""","""grass""",62,63,45,45
"""Venusaur""","""grass""",100,123,16,5
"""Charmander""","""fire""",52,43,45,44
"""Charmeleon""","""fire""",64,58,37,29
"""Charizard""","""fire""",104,78,10,14


#### Range function

In [27]:
# Add an increasing index for the pokemon of each type

pokemon.select(
    'name', 'type1', 'attack', 'defense',
    type_index = pl.arange(1, pl.count()+1).over('type1')
).head(6)

name,type1,attack,defense,type_index
str,str,i64,i64,i64
"""Bulbasaur""","""grass""",49,49,1
"""Ivysaur""","""grass""",62,63,2
"""Venusaur""","""grass""",100,123,3
"""Charmander""","""fire""",52,43,1
"""Charmeleon""","""fire""",64,58,2
"""Charizard""","""fire""",104,78,3


In [28]:
# Take a random sample of 3 pokemons for each group

pokemon.filter(
    pl.arange(1, pl.count()+1).shuffle().over("type1") <= 3
).sort(by='type1').head(6)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary
str,str,str,list[str],i64,i64,i64,i64,i64,i64
"""Sewaddle""","""bug""","""grass""","[""Swarm"", ""Chlorophyll"", ""Overcoat""]",45,53,70,42,5,0
"""Shelmet""","""bug""",null,"[""Hydration"", ""Shell Armor"", ""Overcoat""]",50,40,85,25,5,0
"""Volcarona""","""bug""","""fire""","[""Flame Body"", ""Swarm""]",85,60,65,100,5,0
"""Purrloin""","""dark""",null,"[""Limber"", ""Unburden"", ""Prankster""]",41,50,37,66,5,0
"""Scrafty""","""dark""","""fighting""","[""Shed Skin"", ""Moxie"", ""Intimidate""]",65,90,115,58,5,0
"""Pawniard""","""dark""","""steel""","[""Defiant"", ""Inner Focus"", ""Pressure""]",45,85,70,60,5,0


In [29]:
# Take a random sample of 5% of pokemons for each group

pokemon.filter(
    pl.arange(1, pl.count() + 1).shuffle().over("type1") <= pl.count().over("type1") * 0.05
).sort(by="type1").head(6)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary
str,str,str,list[str],i64,i64,i64,i64,i64,i64
"""Scyther""","""bug""","""flying""","[""Swarm"", ""Technician"", ""Steadfast""]",70,110,80,105,1,0
"""Kricketune""","""bug""",null,"[""Swarm"", ""Technician""]",77,85,51,65,4,0
"""Grubbin""","""bug""",null,"[""Swarm""]",47,62,45,46,7,0
"""Weavile""","""dark""","""ice""","[""Pressure"", ""Pickpocket""]",70,120,65,125,4,0
"""Gible""","""dragon""","""ground""","[""Sand Veil"", ""Rough Skin""]",58,70,45,42,4,0
"""Luxray""","""electric""",null,"[""Rivalry"", ""Intimidate"", ""Guts""]",80,120,79,70,4,0


In [30]:
# Get the 5th pokemon for each group

pokemon.filter(
    pl.arange(1, pl.count() + 1).over("type1") == 5
).head(5)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary
str,str,str,list[str],i64,i64,i64,i64,i64,i64
"""Kakuna""","""bug""","""poison""","[""Shed Skin""]",45,25,50,35,1,0
"""Raticate""","""normal""","""dark""","[""Run Away"", ""Guts"", … ""Thick Fat""]",75,71,70,77,1,0
"""Nidoqueen""","""poison""","""ground""","[""Poison Point"", ""Rivalry"", ""Sheer Force""]",90,92,87,76,1,0
"""Ninetales""","""fire""","""ice""","[""Flash Fire"", ""Drought"", … ""Snow Warning""]",73,67,75,109,1,0
"""Gloom""","""grass""","""poison""","[""Chlorophyll"", ""Stench""]",60,65,70,40,1,0


#### Binning functions: Cut and Qcut

In [31]:
# Bin the attack column into 3 bins
# <= 50 - weak, <= 100 - medium, > 100 - strong

pokemon.get_column('attack').cut([50, 100], labels=['weak', 'medium', 'strong'], maintain_order=True).head(5)

attack,break_point,category
f64,f64,cat
49.0,50.0,"""weak"""
62.0,100.0,"""medium"""
100.0,100.0,"""medium"""
52.0,100.0,"""medium"""
64.0,100.0,"""medium"""


In [32]:
# Assigns the pokemon a category based on their attack stats:
# <= 50 - weak, <= 100 - medium, > 100 - strong

pokemon.with_columns(
    'name', 'type1', 'attack',
    pokemon.get_column('attack').cut([50, 100], labels=['weak', 'medium', 'strong'], maintain_order=True).get_column('category')
).head(6)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary,category
str,str,str,list[str],i64,i64,i64,i64,i64,i64,cat
"""Bulbasaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",45,49,49,45,1,0,"""weak"""
"""Ivysaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",60,62,63,60,1,0,"""medium"""
"""Venusaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",80,100,123,80,1,0,"""medium"""
"""Charmander""","""fire""",null,"[""Blaze"", ""Solar Power""]",39,52,43,65,1,0,"""medium"""
"""Charmeleon""","""fire""",null,"[""Blaze"", ""Solar Power""]",58,64,58,80,1,0,"""medium"""
"""Charizard""","""fire""","""flying""","[""Blaze"", ""Solar Power""]",78,104,78,100,1,0,"""strong"""


In [33]:
# Assigns the pokemon a category based on their attack stats using percentiles:
# bottom 40% - weak, top 20% - strong, rest - medium

pokemon.with_columns(
    'name', 'type1', 'attack',
    pokemon.get_column('attack').qcut(quantiles=[0.4,0.8], labels=['weak', 'medium', 'strong'], maintain_order=True).get_column('category')
).head(6)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary,category
str,str,str,list[str],i64,i64,i64,i64,i64,i64,cat
"""Bulbasaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",45,49,49,45,1,0,"""weak"""
"""Ivysaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",60,62,63,60,1,0,"""weak"""
"""Venusaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",80,100,123,80,1,0,"""medium"""
"""Charmander""","""fire""",null,"[""Blaze"", ""Solar Power""]",39,52,43,65,1,0,"""weak"""
"""Charmeleon""","""fire""",null,"[""Blaze"", ""Solar Power""]",58,64,58,80,1,0,"""weak"""
"""Charizard""","""fire""","""flying""","[""Blaze"", ""Solar Power""]",78,104,78,100,1,0,"""medium"""


#### Repeat

In [34]:
# Use a different dataframe for this example
# Based on a dataframe with 3 fruits and their order quantities
# Repeat the fruit names based on their order quantities

df = pl.DataFrame(
    {
        "Fruit": ["Apple", "Banana", "Cherry"],
        "Order_Quantity": [1, 3, 2],
    }
)

df.select(
    pl.col('Fruit').repeat_by('Order_Quantity').flatten()
    )

Fruit
str
"""Apple"""
"""Banana"""
"""Banana"""
"""Banana"""
"""Cherry"""
"""Cherry"""


#### Explode, Implode and Flatten

In [35]:
# Explode the list of pokemon abilities: from a column of list to a column of values
# Point of attention when using explode: explode increases the number of rows
# All columns in a Polars dataframe should have the same number of rows

pokemon.head(3).select(
    pl.col('abilities').explode()
)

abilities
str
"""Overgrow"""
"""Chlorophyll"""
"""Overgrow"""
"""Chlorophyll"""
"""Overgrow"""
"""Chlorophyll"""


In [36]:
# Implode is the opposite of explode, and aggregates values into lists

pokemon.head(3).select(
    pl.all().implode(),
)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary
list[str],list[str],list[str],list[list[str]],list[i64],list[i64],list[i64],list[i64],list[i64],list[i64]
"[""Bulbasaur"", ""Ivysaur"", ""Venusaur""]","[""grass"", ""grass"", ""grass""]","[""poison"", ""poison"", ""poison""]","[[""Overgrow"", ""Chlorophyll""], [""Overgrow"", ""Chlorophyll""], [""Overgrow"", ""Chlorophyll""]]","[45, 60, 80]","[49, 62, 100]","[49, 63, 123]","[45, 60, 80]","[1, 1, 1]","[0, 0, 0]"


In [37]:
# Flatten removes a level of nesting from a column of lists
# If there is no nesting, then it acts as explodes

pokemon.head(3).select(
    pl.all().implode().flatten()
)

name,type1,type2,abilities,hp,attack,defense,speed,generation,is_legendary
str,str,str,list[str],i64,i64,i64,i64,i64,i64
"""Bulbasaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",45,49,49,45,1,0
"""Ivysaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",60,62,63,60,1,0
"""Venusaur""","""grass""","""poison""","[""Overgrow"", ""Chlorophyll""]",80,100,123,80,1,0


In [38]:
# When there is no nesting, flatten acts as explode

pokemon.head(3).select(
    pl.col('abilities').flatten()
)

abilities
str
"""Overgrow"""
"""Chlorophyll"""
"""Overgrow"""
"""Chlorophyll"""
"""Overgrow"""
"""Chlorophyll"""


#### Conversion between structs and lists

In [39]:
# Combine type 1 and type 2 in a new struct column

pokemon.select(
    'name',
    types = pl.struct('type1', 'type2')
).head(5)

name,types
str,struct[2]
"""Bulbasaur""","{""grass"",""poison""}"
"""Ivysaur""","{""grass"",""poison""}"
"""Venusaur""","{""grass"",""poison""}"
"""Charmander""","{""fire"",null}"
"""Charmeleon""","{""fire"",null}"


In [40]:
# The struct column retains the type information of each initial column:
# Struct are views on the initial columns so the data is not copied
# The table schema shows that the names of the initial columns are retained 

pokemon.select(
    'name',
    types = pl.struct('type1', 'type2')
).schema

{'name': Utf8, 'types': Struct([Field('type1', Utf8), Field('type2', Utf8)])}

In [49]:
# structs are important because they allow us to do calculations on combined columns
# We can calculate the how many times each combination of types occurs

pokemon.select(
    types_count = pl.struct('type1', 'type2').value_counts()
).head(5)

types_count
struct[2]
"{{""grass"",""poison""},14}"
"{{""fire"",null},27}"
"{{""fire"",""flying""},6}"
"{{""water"",null},61}"
"{{""bug"",null},18}"


In [41]:
# Combine type 1 and type 2 in a new list column

pokemon.select(
    'name',
    types = pl.concat_list('type1', 'type2')
).head(5)

name,types
str,list[str]
"""Bulbasaur""","[""grass"", ""poison""]"
"""Ivysaur""","[""grass"", ""poison""]"
"""Venusaur""","[""grass"", ""poison""]"
"""Charmander""","[""fire"", null]"
"""Charmeleon""","[""fire"", null]"


In [42]:
# List columns do not retain the name of the initial columns
# In List columns, each row can have a different number of elements differently from Struct columns
# where each row has the same number of elements

pokemon.select(
    'name',
    types = pl.concat_list('type1', 'type2')
).schema

{'name': Utf8, 'types': List(Utf8)}

In [43]:
# List columns can be converted to a struct
# If number of elements are not the same, then the missing values are filled with null

pokemon.select(
    pl.col('abilities').arr.to_struct(n_field_strategy = 'max_width')
).head(5)

abilities
struct[6]
"{""Overgrow"",""Chlorophyll"",null,null,null,null}"
"{""Overgrow"",""Chlorophyll"",null,null,null,null}"
"{""Overgrow"",""Chlorophyll"",null,null,null,null}"
"{""Blaze"",""Solar Power"",null,null,null,null}"
"{""Blaze"",""Solar Power"",null,null,null,null}"


In [44]:
# By default, the fields inside of the struct are called field_0, field_1, etc.

pokemon.select(
    pl.col('abilities').arr.to_struct(n_field_strategy = 'max_width')
).schema

{'abilities': Struct([Field('field_0', Utf8), Field('field_1', Utf8), Field('field_2', Utf8), Field('field_3', Utf8), Field('field_4', Utf8), Field('field_5', Utf8)])}

In [45]:
# It's possible to change the name with the argument fields, for example calling them ability_0, ability_1, etc.

pokemon.select(
    pl.col('abilities')
    .arr.to_struct(n_field_strategy = 'max_width', fields = lambda i: f'ability_{i}')
).schema

{'abilities': Struct([Field('ability_0', Utf8), Field('ability_1', Utf8), Field('ability_2', Utf8), Field('ability_3', Utf8), Field('ability_4', Utf8), Field('ability_5', Utf8)])}

In [46]:
# We can un-nest struct columns and expand them into multiple columns

pokemon.select(
    pl.col('abilities')
    .arr.to_struct(n_field_strategy = 'max_width')
).unnest('abilities').head(5)

field_0,field_1,field_2,field_3,field_4,field_5
str,str,str,str,str,str
"""Overgrow""","""Chlorophyll""",null,null,null,null
"""Overgrow""","""Chlorophyll""",null,null,null,null
"""Overgrow""","""Chlorophyll""",null,null,null,null
"""Blaze""","""Solar Power""",null,null,null,null
"""Blaze""","""Solar Power""",null,null,null,null


#### Pivot, Melt, Unstack and Transpose

In [58]:
# Pivot type 2 to the columns and count the number of pokemons for each type 1 and type 2 combination

pokemon.pivot(
    values = 'name',
    index = 'type1',
    columns = 'type2',
    aggregate_function=pl.element().count()
)


type1,poison,null,flying,dark,electric,ice,ground,fairy,grass,fighting,psychic,steel,fire,rock,water,dragon,ghost,bug,normal
str,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
"""grass""",14,37,6,3,null,2,1,5,1,3,2,3,null,null,null,null,1,null,null
"""fire""",null,27,6,1,null,2,2,null,null,6,1,1,1,1,1,1,null,null,2
"""water""",3,61,7,4,2,3,9,4,3,2,5,1,null,4,null,2,2,2,null
"""bug""",11,18,13,null,4,null,1,2,6,3,null,5,2,3,3,null,1,null,null
"""normal""",null,61,26,4,null,null,1,4,2,2,3,null,null,null,1,1,null,null,null
"""poison""",2,13,3,3,null,null,2,null,null,2,null,null,2,null,3,1,null,1,null
"""electric""",null,26,3,null,1,null,null,2,null,null,null,4,null,null,null,null,1,null,2
"""ground""",null,10,3,3,1,2,2,null,null,null,2,1,1,3,null,2,2,null,null
"""fairy""",null,16,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [69]:
# Pivot does not exist in lazy mode, however it's possible to reproduce it with group by
# In this case, we need to specify the resulting columns in the code

pokemon.lazy().groupby('type1').agg(
    pl.col('name').filter(pl.col('type2') == pokemon_type).count().alias(pokemon_type)
      for pokemon_type in ['poison', 'flying', 'dark', 'electric']
).collect()

type1,poison,flying,dark,electric
str,u32,u32,u32,u32
"""dragon""",0,4,0,1
"""psychic""",0,6,0,0
"""normal""",0,26,4,0
"""ice""",0,2,0,0
"""rock""",1,4,1,0
"""ground""",0,3,3,1
"""poison""",2,3,3,0
"""grass""",14,6,3,0
"""fighting""",0,1,1,0


In [77]:
# Melt is the opposite of pivot, it brings the header of multiple columns into one column
# and their value in another column

pokemon.melt(
    id_vars = ['name', 'type1', 'type2'],
    value_vars = ['hp','attack', 'defense']
).sort(by='name').head(6)

name,type1,type2,variable,value
str,str,str,str,i64
"""Abomasnow""","""grass""","""ice""","""hp""",90
"""Abomasnow""","""grass""","""ice""","""attack""",132
"""Abomasnow""","""grass""","""ice""","""defense""",105
"""Abra""","""psychic""",null,"""hp""",25
"""Abra""","""psychic""",null,"""attack""",20
"""Abra""","""psychic""",null,"""defense""",15


In [91]:
# Unstack breaks the dataframe into multiple groups of the same size 
# and moves these groups to new columns
# Here we split the dataframe into groups of 3, and add a column with the level of the pokemon

(
    pokemon
    .head(9)
    .unstack(columns = ['name', 'attack', 'defense'], step=3, how="vertical")
    .with_columns(
        level = pl.Series(range(1,4))
        )
)

name_0,name_1,name_2,attack_0,attack_1,attack_2,defense_0,defense_1,defense_2,level
str,str,str,i64,i64,i64,i64,i64,i64,i64
"""Bulbasaur""","""Charmander""","""Squirtle""",49,52,48,49,43,65,1
"""Ivysaur""","""Charmeleon""","""Wartortle""",62,64,63,63,58,80,2
"""Venusaur""","""Charizard""","""Blastoise""",100,104,103,123,78,120,3
